# Pentomino 

总共有12个不同形状的五连体（Pentomino），规定这些形状不能旋转、不能翻转。

需要把这12个形状放进 $6 \times 10, 5\times 12$ 等不同规格的格子里面，每个格子都必须被填上，不能重复。问总共有多少种放置的方法？


⚠️： Unfinished


In [40]:
from typing import Any
from ortools.sat.python import cp_model as cp

class VarArraySolutionCollector(cp.CpSolverSolutionCallback):

    def __init__(self, variables):
        cp.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.solution_list = []
        self.solution_sum = 0


    def on_solution_callback(self):
        self.solution_sum += 1
        print(self.solution_sum)
        for v in self.__variables.keys():
            print(f"{v} : {self.Value(self.__variables[v])}")
        arx = [self.Value(self.__variables[v]) for v in self.__variables.keys()]
        self.solution_list.append(arx)

class PentominoSolver:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.model = cp.CpModel()
        self.solver = cp.CpSolver() 
        self.patterns = "FILNPTUVWXYZ"
        self.x = {}
        self.y = {}
        for i in range(X):
            for j in range(Y):
                for idx in range(len(self.patterns)):
                    self.y[idx, i, j] = self.model.NewBoolVar(f"Fill_{idx}_{i}_{j}")
        
    def solve(self):
        for idx, pattern in enumerate(self.patterns):
            if pattern == "F":
                fill_in_cells = [
                    (0,1), (1,1),(2,1),(2,0),(1,2)
                ]
                not_fill_in_cells = [
                    (0,0), (0,2), (1,2), (2,0)
                ]
                
                current_candidates = []
                for i in range(self.X - 2):
                    for j in range(self.Y - 2):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "I":
                fill_in_cells = [
                    (0,0), (1,0),(2,0),(3,0),(4,0)
                ]
                not_fill_in_cells = []
                current_candidates = []
                
                for i in range(self.X - 4):
                    for j in range(self.Y):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "L":
                fill_in_cells = [
                    (3,0), (3,1),(2,1),(1,1),(0,1)
                ]
                not_fill_in_cells = [
                    (1,1), (2,1), (3,1)
                ]
                current_candidates = []
              
                for i in range(self.X - 3):
                    for j in range(self.Y - 1):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "N":
                fill_in_cells = [
                    (2,0), (3,0), (0,1), (1,1), (2,1)
                ]
                not_fill_in_cells = [
                    (0,1),(1,1),(3,0)
                ]
                current_candidates = []
                
                for i in range(self.X - 3):
                    for j in range(self.Y - 1):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "P":
                fill_in_cells = [
                    (0,0), (0,1), (1,0), (1,1), (2,1)
                ]
                not_fill_in_cells = [
                    (0,1)
                ]
                current_candidates = []
                
                for i in range(self.X - 2):
                    for j in range(self.Y - 1):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "T":
                fill_in_cells = [
                    (0,1),(1,1),(2,1),(2,0),(2,2)
                ]
                not_fill_in_cells = [
                    (0,0), (1,0), (0,2), (1,2)
                ]
                current_candidates = []
               
                for i in range(self.X - 2):
                    for j in range(self.Y - 2):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "U":
                fill_in_cells = [
                    (0,0),(0,1),(1,1),(2,1),(2,0)
                ]
                not_fill_in_cells = [
                    (1,1)
                ]
                current_candidates = []
                
                for i in range(self.X - 2):
                    for j in range(self.Y - 1):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "V":
                fill_in_cells = [
                    (0,0),(0,1),(0,2),(1,0),(2,0)
                ]
                not_fill_in_cells = [
                    (1,1), (1,2), (2,1), (2,2)
                ]
                current_candidates = []
                
                for i in range(self.X - 2):
                    for j in range(self.Y - 2):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "W":
                fill_in_cells = [
                    (0,0), (0,1),(1,1),(1,2),(2,2)
                ]
                not_fill_in_cells = [
                    (0,0), (1,2), (2,1), (2,2)
                ]
                current_candidates = []
                
                for i in range(self.X - 2):
                    for j in range(self.Y - 2):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "X":
                fill_in_cells = [
                    (0,1),(1,1),(1,0), (1,2), (2,1)
                ]
                not_fill_in_cells = [
                    (0,0), (0,2), (2,0), (2,2)
                ]
                current_candidates = []
                
                for i in range(self.X - 2):
                    for j in range(self.Y - 2):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "Y":
                fill_in_cells = [
                    (0,1),(1,1),(2,1), (3,1), (2,0)
                ]
                not_fill_in_cells = [
                    (0,0), (1,0), (3,0)
                ]
                current_candidates = []
                
                for i in range(self.X - 3):
                    for j in range(self.Y - 1):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
            elif pattern == "Z":
                fill_in_cells = [
                    (0,2), (1,0),(1,1),(1,2), (2,0)
                ]
                not_fill_in_cells = [
                    (0,0), (1,0), (1,2), (2,2)
                ]
                current_candidates = []
                
                for i in range(self.X - 2):
                    for j in range(self.Y - 2):
                        self.x[idx, i, j] = self.model.NewBoolVar(f"x[{idx}_{i}_{j}]")
                        current_candidates.append(self.x[idx, i, j])
                        self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in fill_in_cells) == len(fill_in_cells)).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in not_fill_in_cells) == 0).OnlyEnforceIf(self.x[idx, i, j])
                        # self.model.Add(sum(self.y[idx, i + subxx, j + subyy] for (subxx, subyy) in (fill_in_cells + not_fill_in_cells)) == 0).OnlyEnforceIf(self.x[idx, i, j].Not())
                self.model.Add(sum(current_candidates) == 1)
        
        for i in range(self.X):
            for j in range(self.Y):
                # available_pattern = []
                # for idx in range(len(self.patterns)):
                #     if (idx, i, j) in self.x:
                #         available_pattern.append(self.x[idx, i, j])
                self.model.Add(sum(self.y[idx, i,j] for idx in range(12)) == 1)
                    
        solution_collector = VarArraySolutionCollector(self.x)
        self.solver.SearchForAllSolutions(self.model, solution_collector)
        print(solution_collector.solution_sum)
        print(self.solver.SolutionInfo())
        print(self.solver.ResponseStats())
        self.model.ExportToFile("./help.txt")
        # print(len(self.x))

if __name__ == "__main__":
    PentominoSolverTest = PentominoSolver(6, 10)
    PentominoSolverTest.solve()
    
    




0

CpSolverResponse summary:
status: INFEASIBLE
objective: NA
best_bound: NA
integers: 0
booleans: 1025
conflicts: 186506
branches: 399611
propagations: 43041133
integer_propagations: 0
restarts: 2968
lp_iterations: 0
walltime: 11.1309
usertime: 11.1309
deterministic_time: 47.8921
gap_integral: 0



In [6]:
a = [
    (1,2), (2,3)
]
for (x,y) in a:
    print(x, y)

1 2
2 3


In [12]:
a = dict()
a[1,2,3] = 12
print((1,2,3) in a)

True


In [4]:
fill_in_cells = [
        (0,1), (1,0),(1,1),(2,1),(2,2)
    ]
not_fill_in_cells = [
    (0,0), (0,2), (1,2), (2,0)
]
print(fill_in_cells + not_fill_in_cells)

[(0, 1), (1, 0), (1, 1), (2, 1), (2, 2), (0, 0), (0, 2), (1, 2), (2, 0)]
